# Постановка Задачи
### pFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)

В архиве содержится три текстовых файла:

* qid_query.tsv — id запроса и текст запроса, разделённые табуляцией;
* qid_url_rating.tsv — id запроса, URL документа, релевантность документа запросу;
* hostid_url.tsv — id хоста и URL документа.

Нужно вывести текст запроса с максимальным значением метрики , посчитанной по топ-10 документов.

Если для запроса есть несколько документов с одним и тем же id хоста — оставить только максимально релевантный документ (а если несколько документов максимально релевантны, выбрать любой).

Документы по запросу сортируются по убыванию релевантности после выбора одного документа для хоста. Если у нескольких документов с разных хостов релевантность одинакова, их порядок может быть произвольным.

#Yandex pFound with For

In [1]:
import pandas as pd

Залил руками архив на гугл диск для привычного скачивания через gdown

In [2]:
!gdown 1JhkTbsnqsvhlPNPWYRlwkr7EGqDoE3AV

Downloading...
From: https://drive.google.com/uc?id=1JhkTbsnqsvhlPNPWYRlwkr7EGqDoE3AV
To: /content/yandex_cup_analytics_A.zip
100% 75.4k/75.4k [00:00<00:00, 53.9MB/s]


In [3]:
!unzip yandex_cup_analytics_A.zip

Archive:  yandex_cup_analytics_A.zip
replace hidden_task.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: hidden_task.zip         
  inflating: open_task.zip           


In [4]:
!unzip hidden_task.zip

Archive:  hidden_task.zip
replace hostid_url.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: hostid_url.tsv          
  inflating: qid_query.tsv           
  inflating: qid_url_rating.tsv      


In [5]:
!unzip open_task.zip

Archive:  open_task.zip
replace open_task/qid_query.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: open_task/qid_query.tsv  
  inflating: open_task/hostid_url.tsv  
  inflating: open_task/qid_url_rating.tsv  


In [6]:
# считываем данные
qid_query = pd.read_csv("/content/open_task/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("/content/open_task/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("/content/open_task/hostid_url.tsv", sep="\t", names=["hostid", "url"])

In [7]:
# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

In [8]:
def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
    top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ10 урлов с наивысшим рейтингом
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
    return pfound

In [9]:
qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # группируем по qid и вычисляем pfound
qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound

qid_query[qid_query["qid"] == qid_max]

,qid,query
12,295761,гугл переводчик


Найдем средний **pFound** для проверки себя

In [10]:
pfound_mean_orig = qid_pfound.mean()
print(f'Средний pFound: {pfound_mean_orig}')

Средний pFound: 0.4603173929969002


---
# pFoind векторизация

In [11]:
qid_url_rating_hostid.query('qid == 10387').head(10)

,qid,url,rating,hostid
0,10387,http://batman-arkhamcity.ru/,0.00,64
1,10387,http://bigtorrents.org/publ/batman_arkham_city...,0.14,71
2,10387,http://consolelife.ru/xbox-360/6577-o-rossiysk...,0.14,101
3,10387,http://dic.academic.ru/book.nsf/3662736/Batman...,0.00,115
4,10387,http://forum.csmania.ru/viewtopic.php?t=25986,0.14,155
5,10387,http://forum.igromania.ru/printthread.php?s=f3...,0.07,156
6,10387,http://forum.sharereactor.ru/showthread.php?t=...,0.00,161
7,10387,http://forums.4gamers.ru/threads/batman-arkham...,0.07,166
8,10387,http://forums.playground.ru/batman_arkham_city...,0.00,169
9,10387,http://forums.playground.ru/batman_arkham_city...,0.07,169


Согласно условию задачи:
* "Если для запроса есть несколько документов с одним и тем же id хоста — оставить только максимально релевантный документ (а если несколько документов максимально релевантны, выбрать любой)." 

отбросим лишнее

In [12]:
qid_hostid_max_rait = qid_url_rating_hostid.groupby(['qid', 'hostid'])['rating'].max().reset_index()

Проверим

In [13]:
qid_hostid_max_rait.query('qid == 10387').head(10)

,qid,hostid,rating
0,10387,64,0.00
1,10387,71,0.14
2,10387,101,0.14
3,10387,115,0.00
4,10387,155,0.14
5,10387,156,0.07
6,10387,161,0.00
7,10387,166,0.07
8,10387,169,0.07
9,10387,177,0.14


Оставим топ10 хостов для каждого запроса, отсортируем по убыванию

In [14]:
qid_top_10_host = qid_hostid_max_rait.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(10)
qid_top_10_host.head(11)

,qid,hostid,rating
647,380923,6,0.14
654,380923,179,0.14
662,380923,386,0.14
666,380923,484,0.14
665,380923,462,0.07
668,380923,670,0.07
673,380923,1028,0.07
678,380923,1172,0.07
648,380923,45,0.00
649,380923,67,0.00


Обозначим порядковые номера

In [15]:
qid_top_10_host['number'] = qid_top_10_host.groupby('qid').cumcount()

Введем доп множители из формулы, как отдельные колонки
$$A = (1 - pRel)$$
$$B = (1 - pBreak)$$
$$C = A * B$$
$$pBreak = 0.15$$

In [16]:
qid_top_10_host['A'] = (1 - qid_top_10_host['rating']).shift(1) #промежуточный столбец, потом удалим
qid_top_10_host['B'] = 1 - 0.15 #промежуточный столбец, потом удалим
qid_top_10_host.loc[qid_top_10_host['number'] == 0, ['A', 'B']] = 1
qid_top_10_host['C'] = qid_top_10_host['A'] * qid_top_10_host['B']
qid_top_10_host = qid_top_10_host.drop(['A', 'B'], axis=1) #оставим только С для экономии
qid_top_10_host.head(11)

,qid,hostid,rating,number,C
647,380923,6,0.14,0,1.0000
654,380923,179,0.14,1,0.7310
662,380923,386,0.14,2,0.7310
666,380923,484,0.14,3,0.7310
665,380923,462,0.07,4,0.7310
668,380923,670,0.07,5,0.7905
673,380923,1028,0.07,6,0.7905
678,380923,1172,0.07,7,0.7905
648,380923,45,0.00,8,0.7905
649,380923,67,0.00,9,0.8500


Теперь найдем `plook` & `pfound`

In [17]:
qid_top_10_host['plook'] = qid_top_10_host.groupby('qid')['C'].cumprod()
qid_top_10_host['pfound'] = qid_top_10_host['plook'] * qid_top_10_host['rating']
qid_top_10_host.head(11)

,qid,hostid,rating,number,C,plook,pfound
647,380923,6,0.14,0,1.0000,1.000000,0.140000
654,380923,179,0.14,1,0.7310,0.731000,0.102340
662,380923,386,0.14,2,0.7310,0.534361,0.074811
666,380923,484,0.14,3,0.7310,0.390618,0.054687
665,380923,462,0.07,4,0.7310,0.285542,0.019988
668,380923,670,0.07,5,0.7905,0.225721,0.015800
673,380923,1028,0.07,6,0.7905,0.178432,0.012490
678,380923,1172,0.07,7,0.7905,0.141051,0.009874
648,380923,45,0.00,8,0.7905,0.111501,0.000000
649,380923,67,0.00,9,0.8500,0.094775,0.000000


Составим итоговую табличку и рассчитаем среднее `pfound`

In [18]:
pfound_qid = qid_top_10_host.groupby('qid')['pfound'].sum()
pfound_mean = pfound_qid.mean()
print(f'Средний pFound: {pfound_mean}')

Средний pFound: 0.4603173929969002


In [19]:
pfound_mean_orig == pfound_mean

True

Отлично, значения совпали

---
Теперь сделаем функцию

In [20]:
def PFound_mean (df, k):
  '''
  Calculating PFound@k via vectors, where
  'A' = (1 - Rel)
  'B' = (1 - pBreak)
  'C' = A * B
  pBreak = 0,15
  '''

  PBREAK = 0.15
  temp_df = df.copy()
  qid_hostid_max_rel = (temp_df
                        .groupby(['qid', 'hostid'])['rating'].max()
                        .reset_index()
                        )
  qid_k_host = (qid_hostid_max_rel
                       .sort_values(['qid', 'rating'], ascending=False)
                       .groupby(['qid']).head(k)
                       )
  # Additional columns for calculating pfound
  qid_k_host['number'] = qid_k_host.groupby('qid').cumcount()
  qid_k_host['A'] = (1 - qid_k_host['rating']).shift(1) 
  qid_k_host['B'] = 1 - PBREAK
  qid_k_host.loc[qid_k_host['number'] == 0, ['A', 'B']] = 1
  qid_k_host['C'] = qid_k_host['A'] * qid_k_host['B']
  qid_k_host = qid_k_host.drop(['A', 'B'], axis=1)
  # Calculate plook and pfound
  qid_k_host['plook'] = qid_k_host.groupby('qid')['C'].cumprod()
  qid_k_host['pfound'] = qid_k_host['plook'] * qid_k_host['rating']
  # Create 
  pfound_qid = qid_k_host.groupby('qid')['pfound'].sum()
  pfound_mean = pfound_qid.mean()

  return pfound_mean


In [21]:
print(f'Средний PFound: {PFound_mean(qid_url_rating_hostid, 10)}')

Средний PFound: 0.4603173929969002
